# Pipeline per estrarre tutte le tabelle e le immagini da un pdf e poi tutte le immagini e pdf estratti vengono processati per effettuare ocr tramite ollama modello granite. lo fa per tutti i file dentro una cartella

In [ ]:
import logging
import os
import time
from pathlib import Path

from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

_log = logging.getLogger(__name__)

IMAGE_RESOLUTION_SCALE = 2.0

def main():
    logging.basicConfig(level=logging.INFO)

    # cartella PDF in input
    pdf_folder = Path("/storage/data_4T_b/andreacutuli/PROVA/Documents/pdf_fac_simile")
    # cartella radice dove verranno create le sottocartelle di output
    output_root = Path("/storage/data_4T_b/andreacutuli/PROVA/Documents/output_images")

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time_global = time.time()

    # Itera su tutti i PDF nella cartella
    for input_doc_path in pdf_folder.glob("*.pdf"):
        if not input_doc_path.exists():
            _log.warning(f"PDF non trovato: {input_doc_path}")
            continue

        # Crea la sottocartella di output per questo file
        doc_filename = input_doc_path.stem
        output_dir = output_root / doc_filename
        output_dir.mkdir(parents=True, exist_ok=True)

        start_time = time.time()

        conv_res = doc_converter.convert(str(input_doc_path))

        # --- Da qui in poi è TUTTO il tuo codice originale, invariato ---

        # Save page images
        pages_obj = getattr(conv_res.document, "pages", None)
        if pages_obj is None:
            _log.warning("Nessuna proprietà pages trovata su conv_res.document")
        else:
            try:
                iterator = list(pages_obj.items())  # dict-like
                is_dict = True
            except Exception:
                iterator = list(enumerate(pages_obj, start=1))  # list-like
                is_dict = False

            for key, page in iterator:
                page_no = getattr(page, "page_no", None) or (key if is_dict else key)
                page_image_filename = output_dir / f"{doc_filename}-page-{page_no}.png"
                try:
                    page_image = getattr(page, "image", None)
                    if page_image is None:
                        _log.debug(f"Nessuna immagine pagina per page {page_no}")
                        continue
                    pil_img = getattr(page_image, "pil_image", None)
                    if pil_img is not None:
                        pil_img.save(page_image_filename, format="PNG")
                    else:
                        page_image.save(str(page_image_filename), format="PNG")
                except Exception as e:
                    _log.exception(f"Errore salvataggio immagine pagina {page_no}: {e}")

        # Save images of figures and tables
        table_counter = 0
        picture_counter = 0
        try:
            iterator = conv_res.document.iterate_items()
        except Exception:
            iterator = []

        for element, _level in iterator:
            try:
                if isinstance(element, TableItem):
                    table_counter += 1
                    element_image_filename = output_dir / f"{doc_filename}-table-{table_counter}.png"
                    img = element.get_image(conv_res.document)
                    if hasattr(img, "save"):
                        img.save(element_image_filename, format="PNG")
                    else:
                        with open(element_image_filename, "wb") as fp:
                            fp.write(img)
                elif isinstance(element, PictureItem):
                    picture_counter += 1
                    element_image_filename = output_dir / f"{doc_filename}-picture-{picture_counter}.png"
                    img = element.get_image(conv_res.document)
                    if hasattr(img, "save"):
                        img.save(element_image_filename, format="PNG")
                    else:
                        with open(element_image_filename, "wb") as fp:
                            fp.write(img)
            except Exception as e:
                _log.exception(f"Errore salvataggio elemento {type(element)}: {e}")

        # Generazione manuale di markdown con segnaposto
        md_lines = []
        table_counter_md = 0
        picture_counter_md = 0

        try:
            iterator = conv_res.document.iterate_items()
        except Exception:
            iterator = []

        for element, _level in iterator:
            try:
                if isinstance(element, TableItem):
                    table_counter_md += 1
                    md_lines.append(f"[[TABLE-{table_counter_md}]]\n")
                elif isinstance(element, PictureItem):
                    picture_counter_md += 1
                    md_lines.append(f"[[IMAGE-{picture_counter_md}]]\n")
                else:
                    testo = getattr(element, "text", "") or getattr(element, "get_text", lambda: "")()
                    if testo:
                        md_lines.append(testo + "\n")
            except Exception as e:
                _log.exception(f"Errore processando elemento {type(element)}: {e}")

        md_placeholder_filename = output_dir / f"{doc_filename}-with-placeholders.md"
        try:
            with open(md_placeholder_filename, "w", encoding="utf-8") as f:
                f.write("\n".join(md_lines))
            print(f"Markdown con segnaposto salvato in: {md_placeholder_filename}")
        except Exception as e:
            _log.exception(f"Errore salvataggio markdown con segnaposto: {e}")

        end_time = time.time() - start_time
        _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")
        _log.info(f"Saved pages: {page_no if 'page_no' in locals() else 'n/a'}, tables: {table_counter}, pictures: {picture_counter}")

        for nome_file in os.listdir(output_dir):
            percorso_file = os.path.join(output_dir, nome_file)
            if os.path.isfile(percorso_file) and "page" in nome_file:
                os.remove(percorso_file)
                print(f"Cancellato: {nome_file}")

    total_time = time.time() - start_time_global
    _log.info(f"Tutti i PDF elaborati in {total_time:.2f} secondi.")


if __name__ == "__main__":
    main()


In [ ]:
#SPOSTA TUTTI I FILE MARKDOWN IN UNA CARTELLA A PARTE
import os
import shutil

# Cartella sorgente
source_dir = "/storage/data_4T_b/andreacutuli/PROVA/Documents/output_images"

# Cartella di destinazione
dest_dir = "/storage/data_4T_b/andreacutuli/PROVA/Documents/markdown_placeholders"
os.makedirs(dest_dir, exist_ok=True)

# Itera ricorsivamente tutte le sottocartelle
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith(".md"):
            source_path = os.path.join(root, file)
            dest_path = os.path.join(dest_dir, file)

            # Sposta il file (sovrascrive se esiste già)
            shutil.move(source_path, dest_path)
            print(f"Spostato: {source_path} -> {dest_path}")


In [4]:
import os
import ollama

input_root = "/storage/data_4T_b/andreacutuli/PROVA/Documents/output_images"
descrizioni_root = "/storage/data_4T_b/andreacutuli/PROVA/Documents/descrizioni"
os.makedirs(descrizioni_root, exist_ok=True)

# Modello Ollama
model = "granite3.2-vision"
prompt = "Describe what's in this image."

# Itera sulle sottocartelle nella cartella principale di input
for subfolder in os.listdir(input_root):
    subfolder_path = os.path.join(input_root, subfolder)
    if not os.path.isdir(subfolder_path):
        continue  # salta eventuali file direttamente in input_root

    # Crea la sottocartella corrispondente per le descrizioni
    output_subfolder = os.path.join(descrizioni_root, subfolder)
    os.makedirs(output_subfolder, exist_ok=True)

    # Itera su ogni immagine nella sottocartella
    for nome_file in os.listdir(subfolder_path):
        percorso_file = os.path.join(subfolder_path, nome_file)

        if os.path.isfile(percorso_file) and nome_file.lower().endswith((".png", ".jpg", ".jpeg")):
            print(f"\nProcessing image: {nome_file} in folder {subfolder}")

            # Esegui inferenza con Ollama
            response = ollama.generate(
                model=model,
                prompt=prompt,
                images=[percorso_file]
            )

            descrizione = response['response']
            print("Description:", descrizione)

            # Salva la descrizione in file .txt
            txt_filename = os.path.splitext(nome_file)[0] + ".txt"
            txt_path = os.path.join(output_subfolder, txt_filename)
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(descrizione)
            print(f"[OK] Descrizione salvata in {txt_path}")



Processing image: FAC-SIMILE 5-table-1.png in folder FAC-SIMILE 5


2025-10-01 16:52:11,558 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Description: 
The image depicts a table that lists various financial activities along with their respective amounts in euros (€). Here is a detailed description of the table:

### Table Description

**Table Title:** Not explicitly provided, but it appears to be related to financial activities or expenses.

| Descrizione | Importo (€) |
|-----------------|---------------|
| Consolenza strategica finanziaria (Attività 1) | 5.000         |
| Supporto valutazioni progetti (Attività 2) | 6.500         |
| Consolenza finanza agevolata (Attività 3) | 4.500         |
| Totale           | 16.000        |

### Analysis and Description:

The table is structured with two columns: "Descrizione" and "Importo (€)." The "Descrizione" column lists the financial activities or expenses, while the "Importo (€)" column provides the corresponding amounts in euros. 

1. **Consolenza strategica finanziaria (Attività 1):**
   - This activity is described as a "strategic financial consolidation" and has an amou

2025-10-01 16:53:20,211 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Description: 
The image is a table that outlines various financial goals and their corresponding bonus amounts. Here is a detailed description of the table:

### Table Description

**Table Title:** Obiettivo raggiunto

**Columns:**
1. **Obiettivo raggiunto** (Goal Achieved)
2. **Bonus lordo (€)** (Bonus Amount in Euros)
3. **Superamento del 75% di presenza in gare ufficiali** (Achievement of 75% attendance at official games)
4. **10 gol realizzati in Serie A** (10 goals scored in Serie A)
5. **20 gol realizzati in Serie A** (20 goals scored in Serie A)
6. **Superamento del 75% di presenza in gare ufficiali** (Achievement of 75% attendance at official games)
7. **100.000 euro** (100,000 Euros)
8. **100.000 euro** (100,000 Euros)
9. **Superamento del 75% di presenza in gare ufficiali** (Achievement of 75% attendance at official games)
10. **100.000 euro** (100,000 Euros)

### Analysis and Description:

The table is structured to outline different financial goals related to football (socc

...

2025-10-01 18:32:07,276 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Description: 
The image depicts a table with two columns and three rows. The table is structured as follows:

- **First Column**: This column contains textual data, specifically numerical values. Each cell in the first column represents a different value.
- **Second Column**: This column also contains textual data but with an additional "+10%" and "+11%" annotations next to each numerical value. These annotations indicate that the respective values have been increased by 10% and 11%, respectively.

Here is the table converted into markdown format for clarity:

```markdown
| First Column | Second Column |
|----------------|---------------|
| +10%           | +11%          |
| +10%           | +11%          |
| +10%           | +11%          |
| +10%           | +11%          |
```

### Analysis and Description:

The table is structured in a simple manner, with two columns. The first column lists numerical values, while the second column provides additional annotations indicating an incr

2025-10-01 18:32:56,945 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Description: 
The image contains a list of terms related to construction or engineering projects. Here is a detailed description:

1. **Specifiche Tecniche e Normative di Riferimento**
   - This term translates to "Technical Specifications and Reference Standards." It refers to the detailed guidelines, specifications, and standards that are used in construction or engineering projects to ensure quality, safety, and compliance with regulations.

2. **Pianificazione e Fasi di Implementazione**
   - This translates to "Planning and Implementation Phases." It involves breaking down a project into manageable phases and planning the execution of each phase to achieve the desired outcome.

3. **Dettaglio degli Impianti Industriali da Sviluppare**
   - This translates to "Detailed Description of Industrial Plants to be Developed." It involves providing comprehensive information about the design, construction, and operational aspects of industrial plants that are planned for development.

4. **

KeyboardInterrupt: 